In [ ]:
import umap
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

import sys
sys.path.append('/home/diane.kim/ACS/tabular/model/TabPFN/src')
sys.path.append('/home/diane.kim/nature/model/src')

from tabpfn.classifier import TabPFNClassifier
from tabpfn.config import ModelInterfaceConfig
from tabpfn.preprocessing import PreprocessorConfig

from utils.data_preprocessor import *
from utils.fix_seed import *
from models import main, parse_args


In [ ]:
# seed=0

# X_train, X_val, X_test, y_train, y_val, y_test, train_ids, val_ids, test_ids, preprocessor = load_data(binary=True, seed=seed, ref_df="/home/diane.kim/nature/data/final/MASTER_2262_wTTE.csv")

# pfn = init_tabpfn(seed=seed, balance_probabilities=True)
# pfn = train_tabpfn(pfn, X_train, y_train, X_test, y_test)

# emb_train, emb_val, emb_test = get_tab_embeddings(pfn, X_train, X_val, X_test)

# # X = emb_train.cpu().numpy()
# # y = y_train.cpu().numpy()

/home/diane.kim/nature/model/utils/data_preprocessor.py:281: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  ref_df = pd.read_csv(ref_df)


(5851, 40)
Running MANAG vs INTERVENTION.
MANAG
INTERVENTION    4757
MANAG           1094
Name: count, dtype: int64
Label Mapping Used:
MANAG: 0
INTERVENTION: 1
ROC AUC: 0.7242
Accuracy: 0.7063
Balanced Accuracy: 0.6541
F1 Score: 0.7456
Recall: 0.7063
Precision: 0.8203
Class Accuracy:
  0: 0.5811
  1: 0.7271
Spec 20% → Sens: 0.9553, Spec: 0.2027, Thr: 0.3118
Spec 40% → Sens: 0.8949, Spec: 0.4054, Thr: 0.3712
Spec 60% → Sens: 0.7114, Spec: 0.6216, Thr: 0.5274

(4569, 192)
emb_train tensor shape:  torch.Size([4569, 192])


In [ ]:
# # Parameter grid to try
# n_neighbors_list = [5, 15, 50]
# min_dist_list    = [0.0, 0.1, 0.5]
# metric = "cosine"          # "euclidean" also fine; "cosine" often good for embeddings
# random_state = 42

# # Make sure output dir exists
# os.makedirs("plots", exist_ok=True)

# # Create the grid of subplots
# n_rows = len(min_dist_list)
# n_cols = len(n_neighbors_list)
# fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows), squeeze=False)

# for i, md in enumerate(min_dist_list):
#     for j, nn in enumerate(n_neighbors_list):
#         ax = axes[i, j]
#         reducer = umap.UMAP(
#             n_neighbors=nn,
#             min_dist=md,
#             metric=metric,
#             n_components=2,
#             random_state=random_state
#         )
#         X_umap = reducer.fit_transform(X)  # [N, 2]

#         sc = ax.scatter(
#             X_umap[:, 0], X_umap[:, 1],
#             c=y, cmap="coolwarm", s=6, alpha=0.7, linewidths=0
#         )
#         ax.set_title(f"n_neighbors={nn}, min_dist={md}", fontsize=11)
#         ax.set_xticks([]); ax.set_yticks([])

# # One shared colorbar for the whole grid
# cbar = fig.colorbar(sc, ax=axes, shrink=0.7, pad=0.01)
# cbar.set_label("Label (0=MANAG, 1=INTERVENTION)")

# fig.suptitle("UMAP projection of TabPFN embeddings (grid of settings)", y=0.995, fontsize=14)
# plt.tight_layout(rect=[0, 0, 1, 0.98])
# out_path = "plots/umap_tabpfn_grid.png"
# plt.savefig(out_path, dpi=300)
# plt.close()
# print(f"Saved grid figure to: {out_path}")

In [ ]:
# reducer = umap.UMAP(
#     n_neighbors=5,
#     min_dist=0.5,
#     metric="cosine",
#     n_components=3,
#     random_state=seed
# )

# X_umap = reducer.fit_transform(X)  # [N, 3]

# fig = plt.figure(figsize=(8, 6))
# ax = fig.add_subplot(111, projection='3d')

# sc = ax.scatter(
#     X_umap[:, 0], X_umap[:, 1], X_umap[:, 2],
#     c=y, cmap="coolwarm", s=6, alpha=0.7, linewidths=0
# )

# ax.set_title(f"n_neighbors={5}, min_dist={0.5}", fontsize=11)
# ax.set_xlabel("UMAP-1"); ax.set_ylabel("UMAP-2"); ax.set_zlabel("UMAP-3")
# fig.colorbar(sc, ax=ax, shrink=0.75, pad=0.05, label="Label (0=MANAG, 1=INTERVENTION)")

# plt.tight_layout()
# plt.savefig("umap_tabpfn_embeddings_3d.png", dpi=300)
# plt.close()

/home/diane.kim/miniconda3/envs/term2/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/diane.kim/miniconda3/envs/term2/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

seed=0

def collect_hidden_activations(model, X, y, device, batch_size=32, relu_index=1):
    model.eval()
    activation_tensors = []
    labels = []

    handle = model[relu_index].register_forward_hook(
        lambda layer, input, output: activation_tensors.append(output.detach().cpu())
    )
    train_loader = DataLoader(TensorDataset(emb_train, y_train), batch_size=batch_size, sampler=None, shuffle=False)

    with torch.no_grad():
        for x, y in train_loader:
            _ = model(x.to(device))
            labels.append(y.cpu())
    handle.remove()
    activation = torch.cat(activation_tensors, dim=0)
    Y = torch.cat(labels, dim=0)
    return activation.numpy(), Y.numpy()

def umap_and_plot(X_np, y_np, out_png, n_components=3,
                  n_neighbors=5, min_dist=0.5, metric="cosine", seed=seed, title=""):
    reducer = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        metric=metric,
        n_components=n_components,
        random_state=seed
    )
    X_umap = reducer.fit_transform(X_np)

    if n_components == 2:
        plt.figure(figsize=(8,6))
        sc = plt.scatter(X_umap[:,0], X_umap[:,1], c=y_np, cmap="coolwarm", s=6, alpha=0.7)
        plt.colorbar(sc, label="Label (0=MANAG, 1=INTERVENTION)")
        plt.title(title)
        plt.xlabel("UMAP-1"); plt.ylabel("UMAP-2")
        plt.tight_layout(); plt.savefig(out_png, dpi=300); plt.close()
    else:
        from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
        fig = plt.figure(figsize=(8,6))
        ax = fig.add_subplot(111, projection='3d')
        sc = ax.scatter(X_umap[:,0], X_umap[:,1], X_umap[:,2], c=y_np, cmap="coolwarm", s=4, alpha=0.8)
        ax.set_title(title)
        ax.set_xlabel("UMAP-1"); ax.set_ylabel("UMAP-2"); ax.set_zlabel("UMAP-3")
        fig.colorbar(sc, ax=ax, shrink=0.75, pad=0.05, label="Label (0=MANAG, 1=INTERVENTION)")
        plt.tight_layout(); plt.savefig(out_png, dpi=300); plt.close()

X_train, X_val, X_test, y_train, y_val, y_test, train_ids, val_ids, test_ids, preprocessor = load_data(binary=True, seed=seed, ref_df="/home/diane.kim/nature/data/final/MASTER_2262_wTTE.csv")
model, emb_train = train_model(X_train, y_train, X_val, y_val, X_test, y_test, num_epochs=100, optimizer_="SGD", seed=seed, device="cuda")
activation_train, y_train_np = collect_hidden_activations(model, emb_train, y_train, device="cuda")

# umap_and_plot(
#     activation_train, y_train_np,
#     out_png="plots/umap_mlp_train_3d.png",
#     title="UMAP (3D) of MLP hidden activations — train"
# )


/home/diane.kim/nature/model/utils/data_preprocessor.py:281: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  ref_df = pd.read_csv(ref_df)


(5851, 40)
Running MANAG vs INTERVENTION.
MANAG
INTERVENTION    4757
MANAG           1094
Name: count, dtype: int64
Label Mapping Used:
MANAG: 0
INTERVENTION: 1
ROC AUC: 0.7242
Accuracy: 0.7063
Balanced Accuracy: 0.6541
F1 Score: 0.7456
Recall: 0.7063
Precision: 0.8203
Class Accuracy:
  0: 0.5811
  1: 0.7271
Spec 20% → Sens: 0.9553, Spec: 0.2027, Thr: 0.3118
Spec 40% → Sens: 0.8949, Spec: 0.4054, Thr: 0.3712
Spec 60% → Sens: 0.7114, Spec: 0.6216, Thr: 0.5274

(4569, 192)
emb_train tensor shape:  torch.Size([4569, 192])
Epoch 000 | train loss 0.251 | val loss 0.255 | val accuracy 0.604
Epoch 001 | train loss 0.241 | val loss 0.260 | val accuracy 0.648
Epoch 002 | train loss 0.239 | val loss 0.261 | val accuracy 0.469
Epoch 003 | train loss 0.239 | val loss 0.257 | val accuracy 0.604
Epoch 004 | train loss 0.240 | val loss 0.264 | val accuracy 0.424
Epoch 005 | train loss 0.241 | val loss 0.256 | val accuracy 0.582
Epoch 006 | train loss 0.240 | val loss 0.262 | val accuracy 0.512
Epoch 

/home/diane.kim/miniconda3/envs/term2/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/diane.kim/miniconda3/envs/term2/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [3]:
umap_and_plot(
    activation_train, y_train_np,
    n_neighbors=3, min_dist=0.8,
    out_png="plots/umap_mlp_train_3d.png",
    title="UMAP (3D) of MLP hidden activations — train"
)

/home/diane.kim/miniconda3/envs/term2/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/diane.kim/miniconda3/envs/term2/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
